In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
# Regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('Clean_Dataset.csv', index_col=0)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...
300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [3]:
df.stops.unique()

array(['zero', 'one', 'two_or_more'], dtype=object)

In [4]:
dic = {
    'zero': 0,
    'one': 1,
    'two_or_more': 2
}
def convert_stops(df, col_name, dic):
    data = df.copy()
    data[col_name] = data[col_name].map(dic)
    return data


In [5]:
data = df.drop(['flight', 'arrival_time'], axis=1)
data.stops

0         zero
1         zero
2         zero
3         zero
4         zero
          ... 
300148     one
300149     one
300150     one
300151     one
300152     one
Name: stops, Length: 300153, dtype: object

In [6]:
data = convert_stops(data, 'stops', dic)
data

,airline,source_city,departure_time,stops,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,0,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,0,Mumbai,Economy,2.33,1,5953
2,AirAsia,Delhi,Early_Morning,0,Mumbai,Economy,2.17,1,5956
3,Vistara,Delhi,Morning,0,Mumbai,Economy,2.25,1,5955
4,Vistara,Delhi,Morning,0,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...
300148,Vistara,Chennai,Morning,1,Hyderabad,Business,10.08,49,69265
300149,Vistara,Chennai,Afternoon,1,Hyderabad,Business,10.42,49,77105
300150,Vistara,Chennai,Early_Morning,1,Hyderabad,Business,13.83,49,79099
300151,Vistara,Chennai,Early_Morning,1,Hyderabad,Business,10.00,49,81585


In [7]:
column_names = np.array(data.columns)
features = ['airline', 'source_city', 'departure_time', 'destination_city', 'class']
feature_indices = np.where(np.isin(column_names, features))[0]

In [8]:
transformer = ColumnTransformer([('encoder', OneHotEncoder(), feature_indices)],
                                    remainder='passthrough')

In [9]:
transed = transformer.fit_transform(data)
transed.shape

(300153, 30)

In [10]:
data = pd.get_dummies(data)

In [11]:
#Train test split
X = data.drop('price', axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
rm = RandomForestRegressor(n_estimators=20, random_state=123, n_jobs=-1, oob_score=True)
rm.fit(X_train, y_train)

#Output the evaluation metrics of the predictions on the validation set
print("MSE", mean_squared_error(y_test, rm.predict(X_test)))
print("MAE", mean_absolute_error(y_test, rm.predict(X_test)))
print("MAPE", mean_absolute_percentage_error(y_test, rm.predict(X_test)))

/Users/zry/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


MSE 8212327.381729892
MAE 1120.3839529658255
MAPE 0.0720302971353808


/Users/zry/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:550: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
